In [1]:
# import the packages
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import gsw
import shapely.geometry as sg
from shapely import Point


In [2]:
filename = "/scratch/mmurakami/WAOM/categorized.nc"
ds = xr.load_dataset(filename)

In [3]:
ds1 = "/scratch/mmurakami/WAOM/drifter_data_all_withdepth.nc"
ds1 = xr.load_dataset(ds1)

# fill categorized with nan where drifter_data_all is nan

In [4]:
# Apply NaN mask
ds = ds.where(~np.isnan(ds1.Xgrid))

In [5]:
np.where(np.isnan(ds.categorized[-70000].values))[0].shape

(15,)

In [6]:
ds.categorized[-70000]

<xarray.DataArray 'categorized' (drifter: 3895)>
array([4., 1., 4., ..., 5., 1., 1.])
Coordinates:
    time     int64 124353
  * drifter  (drifter) int64 0 1 2 3 4 5 6 ... 3889 3890 3891 3892 3893 3894
Attributes:
    units:        category index
    description:  Categorized water masses

# great, now get the indices where this is DSW at least once

In [7]:
# cols_with_3 = ds.categorized.drif ter[(ds.categorized == 3).any()]

In [8]:
drifters_with_3 = ds.categorized.where(ds.categorized == 3, drop=True).drifter.values

In [9]:
drifters_with_3.shape

(1927,)

# do some value counts to get statistics

In [13]:
arr = ds.categorized[1000]
unique_values, counts = np.unique(arr, return_counts=True)
names = ['AABW', 'MCDW', 'ISW', 'DSW', 'AASW', 'WW', 'mSW','beached']
for i,j in zip(unique_values,counts):
    print(names[int(i)],j,j/3895)

MCDW 1050 0.26957637997432604
ISW 61 0.01566110397946085
DSW 828 0.21258023106546856
AASW 1293 0.3319640564826701
WW 494 0.12682926829268293
mSW 169 0.04338896020539153


In [11]:
y2 = 70080
y4 = 140160
y4_5 = 35040
arr = ds.categorized[y4_5]
unique_values, counts = np.unique(arr, return_counts=True)
names = ['AABW', 'MCDW', 'ISW', 'DSW', 'AASW', 'WW', 'mSW','beached']
beached = 0
for i,j in zip(unique_values,counts):
    if np.isnan(i):
        beached += 1
    else:
        print(names[int(i)],j,(j/3895))
print("beached",beached,beached/3895)

AABW 918 0.2356867779204108
MCDW 926 0.23774069319640564
ISW 167 0.042875481386392814
DSW 21 0.005391527599486521
AASW 1459 0.37458279845956355
WW 375 0.0962772785622593
mSW 29 0.007445442875481386
beached 0 0.0


# show seasonality for these a la figure 2